# Combine CSVs For Processing
Combines the CSVs so all relevant data is on one line and the data is placed based on linking keys through the different CSVs

Everything from this markdown cell to the next one is just a sanity check to make sure the chunking for the CSVs is not giving an odd output. Don't run unless you have some reason to check.

# Combine with Dask

We use dask because it automatically chunks data

We need to determine what features we want to be able to determine which CSVs we need to merge

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

In [2]:
# change to be reflective of your environment
data_dir = '/home/cole/Workspace/School/Capstone/data/first_data_set/TestData/'

In [3]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

In [4]:
# based around linking to BACKUP_OBJECTS
CSV_Merge_List = {
    'NODES' : 'NODEID',
    'FILESPACES' : 'FSID',
    'AF_BITFILES' : ['OBJID', 'BFID'],
    'SD_RECON_ORDER' : 'OBJID',
    'SS_POOLS' : 'POOLID',
    'SD_CHUNK_LOCATIONS' : 'POOLID',}

In [5]:
# AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
# BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])
# ARCHIVE_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/ARCHIVE_OBJECTS.csv"])
# FILESPACES = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/FILESPACES.csv"])
# NODES = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/NODES.csv"])
# SD_CHUNK_COPIES = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_CHUNK_COPIES.csv"])
# SD_CHUNK_LOCATIONS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_CHUNK_LOCATIONS.csv"])
# SD_CONTAINERS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_CONTAINERS.csv"])
# SD_NON_DEDUP_LOCATIONS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/SD_NON_DEDUP_LOCATIONS.csv"])
# SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SD_RECON_ORDER.csv'])
# SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SS_POOLS.csv'])


In [6]:
merge = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])

for key, value in CSV_Merge_List.items():
    print(key, value)
    temp = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/" + key + ".csv"])
    if len(value) == 2:
            merge = merge.join(temp, merge[value[0]] == temp[value[1]], how='left')
    else:        
        merge = merge.join(temp, [value], how='left')

NODES NODEID
FILESPACES FSID
AF_BITFILES ['OBJID', 'BFID']
SD_RECON_ORDER OBJID
SS_POOLS POOLID
SD_CHUNK_LOCATIONS POOLID


In [7]:
df = merge
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))
df.count()

921346

In [8]:
# full_outer_join = BACKUP_OBJECTS
# full_outer_join = full_outer_join.join(NODES, ['NODEID'], how='left')
# full_outer_join = full_outer_join.join(FILESPACES, ['FSID'], how='left')
# full_outer_join = full_outer_join.join(AFBF, full_outer_join.OBJID == AFBF.BFID, how='left')
# full_outer_join = full_outer_join.join(SDRO, ['OBJID'], how='left')
# full_outer_join = full_outer_join.join(SS_POOLS, ['POOLID'], how='left')
# full_outer_join = full_outer_join.join(SD_CHUNK_LOCATIONS, ['POOLID'], how='left')

In [9]:
# df = full_outer_join
# df = df.filter(df.POOLID. isNotNull())
# df = df.withColumn("POOLID", df["POOLID"].cast("int"))
# df = df.filter(df.ATTRLENGTH. isNotNull())
# df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
# df = df.filter(df.BFSIZE. isNotNull())
# df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
# df = df.filter(df.HDRSIZE. isNotNull())
# df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
# df = df.filter(df.OBJID. isNotNull())
# df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [10]:
df.select("OBJID", "ATTRLENGTH", "BFSIZE", "HDRSIZE", "POOLID").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/4_features")

In [11]:
df.count()

921346

In [12]:
new_df = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"//merge_data/4_features/*.csv"])

In [13]:
new_df.count()

921346

AttributeError: 'NoneType' object has no attribute '_jvm'

In [14]:
sc.stop()